In [23]:
import rasterio
print(rasterio.__version__)


1.3.10


In [53]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling


In [55]:
from rasterio.mask import mask


In [57]:
import geopandas as gpd
print(gpd.__version__)

0.14.4


In [59]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Load vaccination site data
vaccination_sites = pd.read_csv(r'C:\Users\HP\OneDrive\Desktop\FlowMinder Assignment\cleaned_vaccination_sites.csv')  
# Convert to GeoDataFrame
gdf_sites = gpd.GeoDataFrame(vaccination_sites, geometry=gpd.points_from_xy(vaccination_sites.longitude, vaccination_sites.latitude))
gdf_sites.set_crs(epsg=4326, inplace=True)  # Set CRS to WGS84

C:\Users\HP\anaconda3\Lib\site-packages\shapely\io.py:105: RuntimeWarning: invalid value encountered in to_wkt
  return lib.to_wkt(


,ID_code,original_name,existing or new,latitude,longitude,target_location,session_type,geometry
0,SOM_existing_194,Baidoa health centre,existing,3.115337,43.652339,Baydhabo,fixed,POINT (43.65234 3.11534)
1,SOM_existing_206,Kurankuro_1,existing,2.895834,43.733333,Baydhabo,mobile,POINT (43.73333 2.89583)
2,SOM_existing_207,Ubor Uuma_1,existing,3.045834,43.511667,Yeed,mobile,POINT (43.51167 3.04583)
3,SOM_existing_208,Warabale_1,existing,3.246667,43.630000,Baydhabo,mobile,POINT (43.63000 3.24667)
4,SOM_existing_209,Daynuunay_1,existing,3.039167,43.799167,Baydhabo,mobile,POINT (43.79917 3.03917)
5,Baydhabo_new_27,Samakaya_1,new,3.032500,43.919167,Baydhabo,mobile,POINT (43.91917 3.03250)
6,NaN,Borama_1,new,2.932500,43.695833,Baydhabo,mobile,POINT (43.69583 2.93250)
7,Baydhabo_new_40,Gaal Looge_1,new,2.975834,43.745000,Baydhabo,mobile,POINT (43.74500 2.97583)
8,Baydhabo_new_50,Gasarta_1,new,3.100000,43.724167,Baydhabo,outreach,POINT (43.72417 3.10000)
9,Baydhabo_new_55,Buur Halab_1,existing,3.105834,43.829167,Baydhabo,mobile,POINT (43.82917 3.10583)


In [61]:
# Check for NaN values in geometry
invalid_geometries = gdf_sites[gdf_sites.geometry.is_empty | gdf_sites.geometry.isna()]
print("Invalid Geometries:\n", invalid_geometries)

# Remove rows with invalid geometries
gdf_sites = gdf_sites[gdf_sites.geometry.notna() & ~gdf_sites.geometry.is_empty]

Invalid Geometries:
 Empty GeoDataFrame
Columns: [ID_code, original_name, existing or new, latitude, longitude, target_location, session_type, geometry]
Index: []


In [63]:
# Remove rows with invalid geometries
gdf_sites = gdf_sites[gdf_sites.geometry.notna() & ~gdf_sites.geometry.is_empty]

# Check if any geometries remain invalid
invalid_geometries_after_removal = gdf_sites[gdf_sites.geometry.is_empty | gdf_sites.geometry.isna()]
if not invalid_geometries_after_removal.empty:
    print("There are still invalid geometries:\n", invalid_geometries_after_removal)
else:
    print("All geometries are valid.")


All geometries are valid.


In [65]:
# Remove rows with invalid geometries or NaN values
gdf_sites = gdf_sites[gdf_sites.geometry.notna() & ~gdf_sites.geometry.is_empty]
gdf_sites = gdf_sites.dropna(subset=['latitude', 'longitude'])

In [67]:
# Reproject to a suitable projected CRS (e.g., EPSG:3395 for global use)
# You may choose a UTM zone suitable for your geographic area.
gdf_sites = gdf_sites.to_crs(epsg=3395)  # Replace with the appropriate EPSG code

In [69]:
# Create a 5 km buffer around each vaccination site (5 km ~ 0.045 degrees)
gdf_sites['buffer'] = gdf_sites.geometry.buffer(0.045)

In [71]:
# Load population rasters (for children aged 0-11 months and 12-59 months)
population_raster_0_11 = r'C:\Users\HP\OneDrive\Desktop\FlowMinder Assignment\Baydhabo_2024_population_0_11m_20240523.tif'  
population_raster_12_59 = r'C:\Users\HP\OneDrive\Desktop\FlowMinder Assignment\Baydhabo_2024_population_12_59m_20240523.tif'  

In [73]:
with rasterio.open(population_raster_0_11) as src:
    raster_crs = src.crs
    print(raster_crs)  # Check raster CRS

# Compare CRS
if gdf_sites.crs != raster_crs:
    print("CRS do not match! Reprojecting...")
    gdf_sites = gdf_sites.to_crs(raster_crs)  # Reproject the GeoDataFrame to match the raster


EPSG:4326
CRS do not match! Reprojecting...


In [77]:
# Output paths for the reprojected rasters
output_raster_0_11 = r'C:\Users\HP\OneDrive\Desktop\FlowMinder Assignment\reprojected_population_0_11.tif'
output_raster_12_59 = r'C:\Users\HP\OneDrive\Desktop\FlowMinder Assignment\reprojected_population_12_59.tif'

# Function to reproject raster
def reproject_raster(input_raster_path, output_raster_path, target_crs):
    with rasterio.open(input_raster_path) as src:
        # Get the source CRS
        source_crs = src.crs
        
        # Calculate the transform and new dimensions
        transform, width, height = calculate_default_transform(
            source_crs,
            target_crs,
            src.width,
            src.height,
            *src.bounds
        )

        # Define the metadata for the new raster
        metadata = src.meta.copy()
        metadata.update({
            'crs': target_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        # Reproject the raster
        with rasterio.open(output_raster_path, 'w', **metadata) as dst:
            for i in range(1, src.count + 1):
                # Reproject each band
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=source_crs,
                    dst_transform=transform,
                    dst_crs=target_crs,
                    resampling=Resampling.nearest  # Choose appropriate resampling method
                )

# Get the target CRS from gdf_sites
target_crs = gdf_sites.crs

# Reproject both rasters
reproject_raster(population_raster_0_11, output_raster_0_11, target_crs)
reproject_raster(population_raster_12_59, output_raster_12_59, target_crs)

print(f"Rasters reprojected and saved to:\n- {output_raster_0_11}\n- {output_raster_12_59}")

Rasters reprojected and saved to:
- C:\Users\HP\OneDrive\Desktop\FlowMinder Assignment\reprojected_population_0_11.tif
- C:\Users\HP\OneDrive\Desktop\FlowMinder Assignment\reprojected_population_12_59.tif


In [85]:
import rasterio

# Define the paths to the reprojected rasters
reprojected_raster_path_0_11 = r'C:\Users\HP\OneDrive\Desktop\FlowMinder Assignment\reprojected_population_0_11.tif'
reprojected_raster_path_12_59 = r'C:\Users\HP\OneDrive\Desktop\FlowMinder Assignment\reprojected_population_12_59.tif'

# Check CRS of the reprojected raster for 0-11 months
with rasterio.open(reprojected_raster_path_0_11) as src:
    crs_0_11 = src.crs
    print(f"CRS of reprojected raster (0-11 months): {crs_0_11}")

# Check CRS of the reprojected raster for 12-59 months
with rasterio.open(reprojected_raster_path_12_59) as src:
    crs_12_59 = src.crs
    print(f"CRS of reprojected raster (12-59 months): {crs_12_59}")


CRS of reprojected raster (0-11 months): EPSG:4326
CRS of reprojected raster (12-59 months): EPSG:4326


In [87]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

# Define the paths for the input and output rasters
population_raster_0_11 = r'C:\Users\HP\OneDrive\Desktop\FlowMinder Assignment\Baydhabo_2024_population_0_11m_20240523.tif'  
population_raster_12_59 = r'C:\Users\HP\OneDrive\Desktop\FlowMinder Assignment\Baydhabo_2024_population_12_59m_20240523.tif'

# Output paths for the reprojected rasters
output_raster_0_11 = r'C:\Users\HP\OneDrive\Desktop\FlowMinder Assignment\reprojected_population_0_11.tif'
output_raster_12_59 = r'C:\Users\HP\OneDrive\Desktop\FlowMinder Assignment\reprojected_population_12_59.tif'

# Define the target CRS
target_crs = 'EPSG:3395'

def reproject_raster(input_raster_path, output_raster_path, target_crs):
    with rasterio.open(input_raster_path) as src:
        # Calculate the transform, width, and height of the new raster
        transform, width, height = calculate_default_transform(
            src.crs, target_crs, src.width, src.height, *src.bounds)

        # Update metadata for the output raster
        metadata = src.meta.copy()
        metadata.update({
            'crs': target_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        # Write the reprojected raster
        with rasterio.open(output_raster_path, 'w', **metadata) as dst:
            for i in range(1, src.count + 1):
                # Reproject each band
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=target_crs,
                    resampling=Resampling.nearest  # Choose appropriate resampling method
                )

# Reproject both rasters
reproject_raster(population_raster_0_11, output_raster_0_11, target_crs)
reproject_raster(population_raster_12_59, output_raster_12_59, target_crs)

print(f"Rasters reprojected to EPSG:3395 and saved to:\n- {output_raster_0_11}\n- {output_raster_12_59}")


Rasters reprojected to EPSG:3395 and saved to:
- C:\Users\HP\OneDrive\Desktop\FlowMinder Assignment\reprojected_population_0_11.tif
- C:\Users\HP\OneDrive\Desktop\FlowMinder Assignment\reprojected_population_12_59.tif


In [139]:
# Initialize a list to store population estimates
population_estimates = []

# Define the paths for the reprojected rasters
reprojected_population_raster_0_11 = r'C:\Users\HP\OneDrive\Desktop\FlowMinder Assignment\reprojected_population_0_11.tif'
reprojected_population_raster_12_59 = r'C:\Users\HP\OneDrive\Desktop\FlowMinder Assignment\reprojected_population_12_59.tif'

# Iterate over each site and its buffer
for index, row in gdf_sites.iterrows():
    total_population = 0
    
    # Process the first reprojected raster (0-11 months)
    with rasterio.open(reprojected_population_raster_0_11) as src:
        # Mask the raster with the buffer of the current site
        out_image, _ = mask(src, [row['buffer']], crop=True)
        # Calculate the sum of the population within the buffer
        total_population += out_image.sum()

    # Process the second reprojected raster (12-59 months)
    with rasterio.open(reprojected_population_raster_12_59) as src:
        # Mask the raster with the buffer of the current site
        out_image, _ = mask(src, [row['buffer']], crop=True)
        # Calculate the sum of the population within the buffer
        total_population += out_image.sum()

    # Append total population for this site to the estimates list
    population_estimates.append(total_population)

# Add estimates to the GeoDataFrame
gdf_sites['population_within_5km'] = population_estimates

# Log the results to a CSV file
results_df = gdf_sites[['ID_code', 'population_within_5km']]
results_df.to_csv(r'C:\Users\HP\population_estimates.csv', index=False)  # Update with your desired output path

In [141]:
print(gdf_sites.crs)
with rasterio.open(population_raster_0_11) as src:
    print(src.crs)


EPSG:3395
EPSG:4326


In [143]:
with rasterio.open(population_raster_0_11) as src:
    print(src.bounds)  # Check the raster bounds
    print(gdf_sites.total_bounds)  # Check the vaccination site buffer bounds


BoundingBox(left=43.101249842, bottom=2.522083525, right=44.310416504, top=3.921250186)
[4843696.61400648  315176.23773734 4889337.60523173  370268.70182795]


In [145]:
no_data_value = src.nodata  # Get the NoData value
population_data = out_image[out_image != no_data_value]  # Ignore NoData areas
total_population = population_data.sum()


In [147]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.enums import Resampling


In [149]:
def reproject_raster(input_raster_path, output_raster_path, target_crs):
    # Open the input raster
    with rasterio.open(input_raster_path) as src:
        # Calculate the transform and dimensions of the output raster
        transform, width, height = calculate_default_transform(
            src.crs, target_crs, src.width, src.height, *src.bounds)
        
        # Define the metadata for the reprojected raster
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': target_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        # Open the output raster and perform the reprojection
        with rasterio.open(output_raster_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=target_crs,
                    resampling=Resampling.nearest)


In [155]:
# Define the CRS of your GeoDataFrame
target_crs = gdf_sites.crs  # This should be the CRS of your GeoDataFrame (e.g., EPSG:3395)

# Reproject the 0-11 months raster
reproject_raster(
    input_raster_path=r'C:\Users\HP\OneDrive\Desktop\FlowMinder Assignment\Baydhabo_2024_population_0_11m_20240523.tif',
    output_raster_path=r'C:\Users\HP\OneDrive\Desktop\Baydhabo_2024_population_0_11m_20240523.tif',
    target_crs=target_crs
)

# Reproject the 12-59 months raster
reproject_raster(
    input_raster_path=r'C:\Users\HP\OneDrive\Desktop\FlowMinder Assignment\Baydhabo_2024_population_12_59m_20240523.tif',
    output_raster_path=r'C:\Users\HP\OneDrive\Desktop\Baydhabo_2024_population_12_59m_20240523.tif',
    target_crs=target_crs
)


In [157]:
# Reprojected population rasters
population_raster_0_11 = r'C:\Users\HP\OneDrive\Desktop\Baydhabo_2024_population_0_11m_20240523.tif'  # Update with the reprojected raster path
population_raster_12_59 = r'C:\Users\HP\OneDrive\Desktop\Baydhabo_2024_population_12_59m_20240523.tif'  # Update with the reprojected raster path

# Initialize a list to store population estimates
population_estimates = []

# Iterate over each site and its buffer
for index, row in gdf_sites.iterrows():
    total_population = 0
    
    # Process the first raster (0-11 months)
    with rasterio.open(population_raster_0_11) as src:
        out_image, _ = mask(src, [row['buffer']], crop=True)
        
        # Handle NoData values (exclude them)
        no_data_value = src.nodata
        population_data = out_image[out_image != no_data_value]
        total_population += population_data.sum()

    # Process the second raster (12-59 months)
    with rasterio.open(population_raster_12_59) as src:
        out_image, _ = mask(src, [row['buffer']], crop=True)
        
        # Handle NoData values (exclude them)
        no_data_value = src.nodata
        population_data = out_image[out_image != no_data_value]
        total_population += population_data.sum()

    # Append total population for this site to the estimates list
    population_estimates.append(total_population)

# Add estimates to the GeoDataFrame
gdf_sites['population_within_5km'] = population_estimates

# Log the results to a CSV file
results_df = gdf_sites[['ID_code', 'population_within_5km']]
results_df.to_csv(r'C:\Users\HP\Desktop\population_estimates.csv', index=False)  # Update with your desired output path

OSError: Cannot save file into a non-existent directory: 'C:\Users\HP\Desktop'